<a href="https://colab.research.google.com/github/Mikhail-068/NetOptic/blob/master/Tretyakov_Aleksandr/2_Downloading_images_to_folders_by_class.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nproc

2


In [ ]:
import gdown
import os

# Загрузка датасета

In [ ]:
DOWNLOAD_LINK = 'https://drive.google.com/uc?id=1Zphx-ht-2p7nUzAg2PuJoirkHh5FvztHvEupL9VLoNs'
base_file_name = 'materials_data.csv'
gdown.download(DOWNLOAD_LINK, output=base_file_name)
file_path = os.path.join('/content', base_file_name)

Access denied with the following error:



 	Cannot retrieve the public link of the file. You may need to change
	the permission to 'Anyone with the link', or have had many accesses. 

You may still be able to access the file from the browser:

	 https://drive.google.com/uc?id=1Zphx-ht-2p7nUzAg2PuJoirkHh5FvztHvEupL9VLoNs 



## Соберём по каждому классу отдельную папку с изображениями

In [ ]:
import os
import pandas as pd
import requests
import time
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

base_file_name = 'materials_data.csv'
file_path = os.path.join('content', base_file_name)

# Скачивание картинок в ```os.cpu_count() - 1``` потоках
# Создание папки materials_pictures, если её ещё нет
if not os.path.exists('materials_pictures'):
    os.makedirs('materials_pictures')

# Создание папок для каждого материала внутри materials_pictures,
# если их ещё нет
base_df = pd.read_csv(file_path)
materials = list(base_df['Материал'].unique())
for material in materials:
    if not os.path.exists(os.path.join('materials_pictures', material)):
        os.makedirs(os.path.join('materials_pictures', material))


# Функция для загрузки картинки
def download_picture(url, save_path):
    try:
        response = requests.get(url)
    except:
        pass
    else:
        if response.status_code == 200:
            with open(save_path, 'wb') as f:
                f.write(response.content)
        else:
            print(f'Ошибка загрузки изображения: {url}')

    # Добавление объекта tqdm для отслеживания прогресса загрузки
    pbar.update(1)


# Проход по строкам DataFrame и скачивание картинок
start_time = time.time()
with ThreadPoolExecutor(max_workers=os.cpu_count() - 1) as executor:
    futures = []
    for index, row in base_df.iterrows():
        material = row['Материал']
        picture_links = row['picture'].split('\n')
        for link in picture_links:
            filename = os.path.join('materials_pictures', material, os.path.basename(link)).replace('%20', '_')
            futures.append(executor.submit(download_picture, link, filename))

    # Инициализация объекта tqdm
    with tqdm(total=len(futures)) as pbar:
        for future in as_completed(futures):
            future.result()

# Вывод времени выполнения
end_time = time.time()
print(f'Время выполнения: {end_time - start_time:.2f} секунд')

In [ ]:
import shutil

# Создание архива папки materials_pictures
result = shutil.make_archive('materials_pictures_archive', 'zip',
                             'materials_pictures')
print(f'Архив картинок скачать здесь:\n{result}')